In [1]:
from binance.um_futures import UMFutures
# import gym
# import json

import time
import datetime
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler

pd.options.mode.copy_on_write = True

In [2]:
# Test Net API Key and Secret
API_KEY = "b88d219ce4b7f4408ab0f51fc8be1730b264904e48e9a6ac7c7fa716e1e18dc7"
API_SECRET = "15acdeca3bb1dd761e472034c8b09bb597e165d18c69e485738ccf9a3aa4326a"

In [3]:
base_url_REST_testnet = "https://testnet.binancefuture.com"
base_url_WS_testnet = "wss://stream.binancefuture.com"

In [4]:
um_futures_client = UMFutures(key=API_KEY, secret=API_SECRET, base_url=base_url_REST_testnet)

In [5]:
def get_data(client,
             symbol,
             interval: str="1h",
             startTime: int=None,
             endTime: int=None
            ):
    if startTime is None or endTime is None:
        data = client.klines(
            limit=5, # to be removed
            symbol=symbol,
            interval=interval
        )
    else:
        data = client.klines(
            symbol=symbol,
            interval=interval,
            limit=1500,
            startTime=startTime,
            endTime=endTime
        )
    df = pd.DataFrame(data).iloc[:, :6]
    columns = [
        "Open Time",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume"
    ]
    df.columns = columns
    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    df.set_index('Open Time', inplace=True)
    columns_float = [
        "Open",
        "High",
        "Low",
        "Close",
        "Volume"
    ]
    df[columns_float] = df[columns_float].astype(float)
    return df

In [6]:
def cal_rolling_zScore(df,
                       window_size: int=14):
    # Calculate rolling mean and rolling standard deviation
    df['Rolling Mean'] = df['Close'].rolling(window=window_size).mean()
    df['Rolling Std']  = df['Close'].rolling(window=window_size).std()
    
    # Calculate the rolling z-score
    df['Rolling Z Score'] = (df['Close'] - df['Rolling Mean']) / df['Rolling Std']
    return df.dropna()

In [7]:
symbols = ['BTCUSDT', 'ETHUSDT']

## Analysis

In [8]:
# startTime = datetime.datetime(2024, 1, 1, 0, 0)
# endTime = datetime.datetime(2024, 9, 30, 0, 0)

# data = {}

# for symbol in symbols:
#     for interval in intervals:
#         data[symbol + "_" + interval] = get_data(client=um_futures_client,
#                                                  symbol=symbol,
#                                                  interval=interval,
#                                                  startTime=int(startTime.timestamp() * 1000),
#                                                  endTime=int(endTime.timestamp() * 1000))

In [9]:
# for d in data:
#     data[d] = cal_rolling_zScore(data[d])

In [10]:
# diffs = {}

# for interval in intervals:
#     asset1 = 'BTCUSDT_' + interval
#     asset2 = 'ETHUSDT_' + interval
#     price_ratios = (data[asset1]['Close'] / data[asset2]['Close']).values
#     scaler = StandardScaler().fit(price_ratios.reshape(-1, 1))
#     z_scores = scaler.transform(price_ratios.reshape(-1, 1)).flatten()
#     diffs[interval] = pd.DataFrame({
#         'Open Time': data[asset1]['Open Time'],
#         'Z Diff': z_scores
#         })

In [11]:
# plt.figure(figsize=(15,5))
# for idx, diff in enumerate(diffs):
#     plt.subplot(1, len(diffs), idx+1)
#     plt.plot(diffs[diff]['Open Time'],
#              diffs[diff]['Z Diff'])
#     plt.title(diff)

# # Adding labels and title
# # plt.xlabel('Time')
# # plt.ylabel('Price')
# # plt.title('Comparison of Values from Two DataFrames')
# # plt.legend()
# # plt.grid()

# # Show the plot
# plt.show()

## Choose 5m as trading interval

In [12]:
data = {}
for symbol in symbols:
    df = None
    startTime = datetime.datetime(2024, 1, 1, 0, 0)
    while startTime < datetime.datetime(2024, 10, 1, 0, 0):
        endTime = startTime + datetime.timedelta(days=5)
        df_new = get_data(client=um_futures_client,
                            symbol=symbol,
                            interval='5m',
                            startTime=int(startTime.timestamp() * 1000),
                            endTime=int(endTime.timestamp() * 1000))
        if df is None:
            df = df_new
        else:
            df = pd.concat([df, df_new])
        startTime = endTime + datetime.timedelta(minutes=5)
    data[symbol] = df

In [13]:
for symbol in symbols:
    data[symbol] = cal_rolling_zScore(data[symbol])

In [14]:
spread_df = pd.DataFrame(index=data['BTCUSDT'].index.intersection(data['ETHUSDT'].index))

# k is the ratio between BTC and ETH
# To calculate the adjusted spread, the formula is
# Z_X - k * Z_Y for assets X and Y

spread_df['k'] = (data['ETHUSDT']['Rolling Std'] / data['BTCUSDT']['Rolling Std']).clip(4, 0)
spread_df['Adjusted Spread'] = data['ETHUSDT']['Rolling Z Score'] - spread_df['k'] * data['BTCUSDT']['Rolling Z Score']

# When the Spread is Positive, it means that BTC is overpriced.
# Therefore, the strategy is short BTC and long ETH

## Simulation

In [15]:
cash = 10000
leverage = 5

In [16]:
class Position():
    def __init__(self):
        self.quantity = 0
        self.entry_price = 0
        self.notional_size = 0
        self.leverage = 0
        self.margin = 0
        
    def open_position(self,
                      quantity: float,
                      price: float,
                      leverage: int):
        self.quantity = quantity
        self.entry_price = price
        self.notional_size = abs(quantity * price)
        self.leverage = leverage
        self.margin = self.notional_size / leverage

    def __str__(self):
        return f"""
        Quantity = {self.quantity}
        Entry Price = {self.entry_price}
        Notional Size = {self.notional_size}
        Leverage = {self.leverage}
        Margin = {self.margin}
        """
    
    def close_position(self, price):
        pnl = self.get_pnl(price)
        self.reset()
        return pnl
    
    def get_pnl(self, price):
        return (price - self.entry_price) * self.quantity
    
    def reset(self):
        self.quantity = 0
        self.entry_price = 0
        self.notional_size = 0
        self.leverage = 0
        self.margin = 0        

In [17]:
class Account():
    def __init__(self,
                 positions,
                 cash: int=10000):
        self.cash = cash
        self.status = None
        self.margin_balance = 0
        self.margin_ratio = 0
        self.pnl = 0
        self.positions = positions

    def __str__(self):
        return f"""
        Cash = {self.cash}
        Status = {self.status}
        BTC Quantity = {self.positions['BTC'].quantity}
        BTC Entry Price = {self.positions['BTC'].entry_price}
        ETH Quantity = {self.positions['ETH'].quantity}
        ETH Entry Price = {self.positions['ETH'].entry_price}
        """
    
    def open_position(self,
                      status: str,
                      asset_1: str,
                      asset_2: str,
                      price_asset_1: float,
                      price_asset_2: float,
                      leverage: int,
                      k: float):
        self.status = status
        unit_cost = price_asset_1 + k * price_asset_2
        quantity_asset_1 = self.cash * leverage / unit_cost
        quantity_asset_2 = self.cash * leverage * k / unit_cost
        
        if status == 'Long':
            self.positions[asset_1].open_position(-quantity_asset_1, price_asset_1, leverage)
            self.positions[asset_2].open_position(quantity_asset_2, price_asset_2, leverage)
        if status == 'Short':
            self.positions[asset_1].open_position(quantity_asset_1, price_asset_1, leverage)
            self.positions[asset_2].open_position(-quantity_asset_2, price_asset_2, leverage)

    def close_position(self,
                       asset_1,
                       asset_2,
                       price_asset_1,
                       price_asset_2):
        self.status = None
        self.cash += self.positions[asset_1].close_position(price_asset_1)
        self.cash += self.positions[asset_2].close_position(price_asset_2)

In [28]:
ETH = Position()
BTC = Position()
positions = {'ETH': ETH,
             'BTC': BTC}
account = Account(positions=positions, cash=cash)

for i, row in spread_df.iterrows():
    spread = row['Adjusted Spread']
    k = row['k']
    price_asset_1 = data['ETHUSDT'].loc[i, 'Close']
    price_asset_2 = data['BTCUSDT'].loc[i, 'Close']

    if account.status is None:
        if spread < -2:
            params = {
                'status': 'Short',
                'asset_1': 'ETH',
                'asset_2': 'BTC',
                'price_asset_1': price_asset_1,
                'price_asset_2': price_asset_2,
                'leverage': leverage,
                'k': k
            }
            account.open_position(**params)
        if spread > 2:
            params = {
                'status': 'Long',
                'asset_1': 'ETH',
                'asset_2': 'BTC',
                'price_asset_1': price_asset_1,
                'price_asset_2': price_asset_2,
                'leverage': leverage,
                'k': k
            }
            account.open_position(**params)
    elif account.status == 'Long':
        if spread < 0:
            params = {
                'asset_1': 'ETH',
                'asset_2': 'BTC',
                'price_asset_1': price_asset_1,
                'price_asset_2': price_asset_2
            }
            account.close_position(**params)
    elif account.status == 'Short':
        if spread > 0:
            params = {
                'asset_1': 'ETH',
                'asset_2': 'BTC',
                'price_asset_1': price_asset_1,
                'price_asset_2': price_asset_2
            }
            account.close_position(**params)

In [29]:
print(account)


        Cash = 1.672416718466222e+32
        Status = None
        BTC Quantity = 0
        BTC Entry Price = 0
        ETH Quantity = 0
        ETH Entry Price = 0
        


## Trading Futures with API

In [20]:
# symbol = 'ETHUSDT'
# leverage = 20

In [21]:
# # Change leverage
# params = {
#     'symbol': symbol,
#     'leverage': leverage
# }

# response = um_futures_client.change_leverage(**params)
# print(response)

In [22]:
# # Post a new order
# params = {
#     'symbol': symbol,
#     'side': 'SELL',
#     'type': 'LIMIT',
#     'timeInForce': 'GTC',
#     'quantity': 25,
#     'price': 2540
# }

# response = um_futures_client.new_order(**params)

In [23]:
# params = {
#     'symbol': symbol,
#     'side': 'BUY',
#     'type': 'MARKET',
#     'quantity': 1
# }

# response = um_futures_client.new_order(**params)
# print(printable_json(response))

In [24]:
# params = {
#     'symbol': symbol,
#     'side': 'SELL',
#     'type': 'STOP_MARKET',
#     'quantity': 1,
#     'stopPrice': 2600
# }

# response = um_futures_client.new_order(**params)
# print(printable_json(response))

In [25]:
# # Post a new order
# params = {
#     'symbol': symbol
# }

# response = um_futures_client.get_position_margin_history(**params)
# print(printable_json(response))

In [26]:
# response = um_futures_client.get_position_risk(**params)
# print(printable_json(response))

In [27]:
# def printable_json(response):
#     return json.dumps(response, indent = 4)